In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
from PIL import Image
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 파일이 담긴 폴더 경로
folder_path = "/content/drive/MyDrive/OX/Dataset"

# 파일 리스트 가져오기
file_names = os.listdir(folder_path)

# 이미지 파일 리스트
image_files = [file for file in file_names if file.lower().endswith('.jpg') or file.lower().endswith('.png')]

# 라벨 추출
labels = []

for file_name in image_files:
    if 'o' in file_name.lower():
        labels.append(1)
    elif 'x' in file_name.lower():
        labels.append(0)

# 이미지 데이터를 numpy 배열로 변환
image_data = []

for file_name in image_files:
    image_path = os.path.join(folder_path, file_name)
    image = Image.open(image_path).convert("L")  # 그레이스케일로 변환
    image = image.resize((300, 300))  # 이미지 크기를 300x300으로 조정
    image_array = np.array(image)  # 이미지를 numpy 배열로 변환
    image_data.append(image_array)

image_data = np.array(image_data)


labels = np.array(labels)

labels = to_categorical(labels, num_classes=2)

from sklearn.model_selection import train_test_split

x = image_data
y = labels

In [ ]:
# mish swish activation
import keras.backend as K
from keras.layers import Activation

def mish(x):
    return x * K.tanh(K.softplus(x))

def swish(x):
    return x * K.sigmoid(x)

mish_activation = Activation(mish)
swish_activation = Activation(swish)

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import mean_squared_error, accuracy_score

# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(image_data, labels, test_size=0.2, random_state=42)

# Leave-One-Out Cross-Validation
fold_rmse = []
fold_acc = []
for i in range(len(x_train)):
    x_train = np.concatenate([x[:i], x[i+1:]], axis=0)
    x_test = x[i:i+1]
    y_train = np.concatenate([y[:i], y[i+1:]], axis=0)
    y_test = y[i:i+1]

    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(300, 300, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # 이진 분류를 위한 출력 레이어

    model.compile(optimizer='adam', loss='mean_squared_error')

    history = model.fit(x_train, y_train, epochs=2, batch_size=5, validation_split=0.2, verbose=2)

    predictions = model.predict(x_test)
    # 클래스로 변환
    predictions_classes = np.round(predictions).astype(int)
    acc = accuracy_score(y_test, predictions_classes)
    fold_acc.append(acc)

    print(f'Fold {i} Accuracy: {acc}\n')

print(f'Average Accuracy across folds: {np.mean(fold_acc)}')

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

# 데이터 분할
kf = KFold(n_splits=10, shuffle=True, random_state=42)
fold_acc = []

# K-Fold Cross Validation
for fold, (train_index, test_index) in enumerate(kf.split(x), 1):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # CNN 모델 생성
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(300, 300, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # 이진 분류를 위한 출력 레이어

    # 모델 컴파일
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

    # 모델 훈련
    model.fit(x_train, y_train, epochs=4, batch_size=5, validation_split=0.2, verbose=2)

        # 모델 평가 후 분류 결과 출력
    predictions = model.predict(x_test)
    predictions_classes = np.round(predictions).astype(int)
    acc = accuracy_score(y_test, predictions_classes)

    for i, (pred_class, actual_class) in enumerate(zip(predictions_classes, y_test)):
        print(f'Fold {fold} Sample {i+1} - Predicted Class: {pred_class}, Actual Class: {actual_class}')
    print(f'Fold {fold} Accuracy: {acc}')

# 모든 폴드의 평균 정확도 출력
print(f'Average Accuracy across folds: {np.mean(fold_acc)}')


Epoch 1/4


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

# 데이터 분할
kf = KFold(n_splits=3, shuffle=True, random_state=42)
fold_acc = []

# K-Fold Cross Validation
for fold, (train_index, test_index) in enumerate(kf.split(x), 1):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # CNN 모델 생성
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(300, 300, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    # 모델 컴파일
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # 모델 훈련
    model.fit(x_train, y_train, epochs=5, batch_size=5, validation_split=0.2, verbose=2)

        # 모델 평가 후 분류 결과 출력
    predictions = model.predict(x_test)
    predictions_classes = np.argmax(predictions, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)
    acc = accuracy_score(y_test_classes, predictions_classes)

    for i, (pred_class, actual_class) in enumerate(zip(predictions_classes, y_test_classes)):
        print(f'Fold {fold} Sample {i+1} - Predicted Class: {pred_class}, Actual Class: {actual_class}')
    print(f'Fold {fold} Accuracy: {acc}')
    fold_acc.append(acc)
# 모든 폴드의 평균 정확도 출력
print(f'Average Accuracy across folds: {np.mean(fold_acc)}')


Epoch 1/5
9/9 - 3s - loss: 25015.2090 - accuracy: 0.5714 - val_loss: 1919.0973 - val_accuracy: 0.4545 - 3s/epoch - 286ms/step
Epoch 2/5
9/9 - 0s - loss: 237.4520 - accuracy: 0.5238 - val_loss: 0.6657 - val_accuracy: 0.5455 - 199ms/epoch - 22ms/step
Epoch 3/5
9/9 - 0s - loss: 0.6934 - accuracy: 0.4286 - val_loss: 0.6568 - val_accuracy: 0.6364 - 197ms/epoch - 22ms/step
Epoch 4/5
9/9 - 0s - loss: 0.6932 - accuracy: 0.5238 - val_loss: 0.6349 - val_accuracy: 0.5455 - 198ms/epoch - 22ms/step
Epoch 5/5
9/9 - 0s - loss: 0.6931 - accuracy: 0.5476 - val_loss: 0.6272 - val_accuracy: 0.5455 - 198ms/epoch - 22ms/step
1/1 [==============================] - 0s 306ms/step
Fold 1 Sample 1 - Predicted Class: 1, Actual Class: 0
Fold 1 Sample 2 - Predicted Class: 1, Actual Class: 0
Fold 1 Sample 3 - Predicted Class: 1, Actual Class: 1
Fold 1 Sample 4 - Predicted Class: 1, Actual Class: 0
Fold 1 Sample 5 - Predicted Class: 1, Actual Class: 0
Fold 1 Sample 6 - Predicted Class: 1, Actual Class: 0
Fold 1 Samp

In [ ]:
!pip install tensorflow tensorflow-addons


In [ ]:
!pip install --upgrade tensorflow tensorflow-addons


In [ ]:
import tensorflow_addons as tfa

# 데이터 분할
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_acc = []

# K-Fold Cross Validation
for fold, (train_index, test_index) in enumerate(kf.split(x), 1):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # CNN 모델 생성
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='tfa.activations.mish', input_shape=(300, 300, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='tfa.activations.mish'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='tfa.activations.mish'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    # 모델 컴파일
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # 모델 훈련
    model.fit(x_train, y_train, epochs=20, batch_size=5, validation_split=0.2, verbose=2)

    predictions = model.predict(x_test)
    predictions_classes = np.argmax(predictions, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)
    acc = accuracy_score(y_test_classes, predictions_classes)

    for i, (pred_class, actual_class) in enumerate(zip(predictions_classes, y_test_classes)):
        print(f'Fold {fold} Sample {i+1} - Predicted Class: {pred_class}, Actual Class: {actual_class}')
    print(f'Fold {fold} Accuracy: {acc}')
    fold_acc.append(acc)

# 모든 폴드의 평균 정확도 출력
print(f'Average Accuracy across folds: {np.mean(fold_acc)}')

ModuleNotFoundError: No module named 'tensorflow_addons'

In [ ]:
# 데이터 분할
kf = KFold(n_splits=10, shuffle=True, random_state=42)
fold_acc = []

# K-Fold Cross Validation
for fold, (train_index, test_index) in enumerate(kf.split(x), 1):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # CNN 모델 생성
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='mish', input_shape=(300, 300, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='mish'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='mish'))
    model.add(MaxPooling2D((2, 2)))
    # model.add(Conv2D(256, (3, 3), activation='mish'))
    # model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='mish'))
    model.add(Dense(256, activation='mish'))
    model.add(Dense(2, activation='softmax'))

    # 모델 컴파일
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # 모델 훈련
    model.fit(x_train, y_train, epochs=15, batch_size=10, validation_split=0.2, verbose=2)

    predictions = model.predict(x_test)
    predictions_classes = np.argmax(predictions, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)
    acc = accuracy_score(y_test_classes, predictions_classes)

    for i, (pred_class, actual_class) in enumerate(zip(predictions_classes, y_test_classes)):
        print(f'Fold {fold} Sample {i+1} - Predicted Class: {pred_class}, Actual Class: {actual_class}')
    print(f'Fold {fold} Accuracy: {acc}')
    fold_acc.append(acc)

# 모든 폴드의 평균 정확도 출력
print(f'Average Accuracy across folds: {np.mean(fold_acc)}')

Epoch 1/15
6/6 - 2s - loss: 272.8976 - accuracy: 0.5263 - val_loss: 18.7068 - val_accuracy: 0.5333 - 2s/epoch - 406ms/step
Epoch 2/15
6/6 - 0s - loss: 13.2369 - accuracy: 0.5614 - val_loss: 1.4989 - val_accuracy: 0.7333 - 344ms/epoch - 57ms/step
Epoch 3/15
6/6 - 0s - loss: 3.1216 - accuracy: 0.8070 - val_loss: 11.1232 - val_accuracy: 0.6667 - 345ms/epoch - 58ms/step
Epoch 4/15
6/6 - 0s - loss: 4.8038 - accuracy: 0.8070 - val_loss: 18.6238 - val_accuracy: 0.6667 - 308ms/epoch - 51ms/step
Epoch 5/15
6/6 - 0s - loss: 2.6525 - accuracy: 0.8947 - val_loss: 13.7112 - val_accuracy: 0.6000 - 344ms/epoch - 57ms/step
Epoch 6/15
6/6 - 0s - loss: 0.0386 - accuracy: 0.9825 - val_loss: 15.6124 - val_accuracy: 0.6000 - 310ms/epoch - 52ms/step
Epoch 7/15
6/6 - 0s - loss: 0.0023 - accuracy: 1.0000 - val_loss: 23.9503 - val_accuracy: 0.6000 - 311ms/epoch - 52ms/step
Epoch 8/15
6/6 - 0s - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 25.8841 - val_accuracy: 0.6000 - 311ms/epoch - 52ms/step
Epoch 9/15
6

In [ ]:
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

# 파일이 담긴 폴더 경로
folder_path = "/content/drive/MyDrive/OX/Dataset"

# 파일 리스트 가져오기
file_names = os.listdir(folder_path)

# 이미지 파일 리스트
image_files = [file for file in file_names if file.lower().endswith('.jpg') or file.lower().endswith('.png')]

# 라벨 추출
labels = []

for file_name in image_files:
    if 'o' in file_name.lower():
        labels.append(1)
    elif 'x' in file_name.lower():
        labels.append(0)

# 이미지 데이터를 numpy 배열로 변환
image_data = []

for file_name in image_files:
    image_path = os.path.join(folder_path, file_name)
    image = Image.open(image_path).convert("L")  # 그레이스케일로 변환
    image = image.resize((300, 300))  # 이미지 크기를 300x300으로 조정
    image_array = np.array(image) / 255.0  # 이미지를 numpy 배열로 변환
    image_data.append(image_array)

image_data = np.array(image_data)

# 라벨을 numpy 배열로 변환
labels = np.array(labels)
labels = to_categorical(labels, num_classes=2)

x = image_data
y = labels

# 데이터 분할
kf = KFold(n_splits=10, shuffle=True, random_state=42)
fold_acc = []

# K-Fold Cross Validation
for fold, (train_index, test_index) in enumerate(kf.split(x), 1):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    test_file_names = [image_files[i] for i in test_index]  # 테스트 셋의 파일 이름 추출

    # CNN 모델 생성
    model = Sequential()
    model.add(Conv2D(32, (4, 4), activation='mish', input_shape=(300, 300, 1)))
    model.add(MaxPooling2D((3, 3)))
    model.add(Conv2D(64, (3, 3), activation='mish'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='mish'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(256, (3, 3), activation='mish'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(512, (3, 3), activation='mish'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation=mish))
    model.add(Dense(2, activation='softmax'))

    # 모델 컴파일
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # 모델 훈련
    model.fit(x_train, y_train, epochs=20, batch_size=5, validation_split=0.2, verbose=2)

    # 모델 평가
    predictions = model.predict(x_test)
    predictions_classes = np.argmax(predictions, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)
    acc = accuracy_score(y_test_classes, predictions_classes)

    # 분류 결과 출력
    for file_name, pred_class, actual_class in zip(test_file_names, predictions_classes, y_test_classes):
        print(f'Fold {fold} - File: {file_name}, Predicted Class: {pred_class}, Actual Class: {actual_class}')
    print(f'Fold {fold} Accuracy: {acc}')
    fold_acc.append(acc)

# 모든 폴드의 평균 정확도 출력
print(f'Average Accuracy across folds: {np.mean(fold_acc)}')


Epoch 1/20
12/12 - 17s - loss: 1.1572 - accuracy: 0.5614 - val_loss: 0.7754 - val_accuracy: 0.5333 - 17s/epoch - 1s/step
Epoch 2/20
12/12 - 14s - loss: 0.7410 - accuracy: 0.5263 - val_loss: 0.5994 - val_accuracy: 0.6000 - 14s/epoch - 1s/step
Epoch 3/20
12/12 - 16s - loss: 0.4726 - accuracy: 0.8421 - val_loss: 0.6401 - val_accuracy: 0.6667 - 16s/epoch - 1s/step
Epoch 4/20
12/12 - 13s - loss: 0.4213 - accuracy: 0.8070 - val_loss: 0.6482 - val_accuracy: 0.6667 - 13s/epoch - 1s/step
Epoch 5/20
12/12 - 12s - loss: 0.2896 - accuracy: 0.8246 - val_loss: 0.7443 - val_accuracy: 0.6667 - 12s/epoch - 1s/step
Epoch 6/20
12/12 - 14s - loss: 0.1963 - accuracy: 0.9474 - val_loss: 1.1519 - val_accuracy: 0.6000 - 14s/epoch - 1s/step
Epoch 7/20
12/12 - 14s - loss: 0.2025 - accuracy: 0.8947 - val_loss: 0.6419 - val_accuracy: 0.6000 - 14s/epoch - 1s/step
Epoch 8/20
12/12 - 14s - loss: 0.1441 - accuracy: 0.9474 - val_loss: 0.7761 - val_accuracy: 0.6667 - 14s/epoch - 1s/step
Epoch 9/20
12/12 - 14s - loss: 0

### Loocv

In [ ]:
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut

# Leave-One-Out Cross Validation
loo = LeaveOneOut()
fold_acc = []

for fold, (train_index, test_index) in enumerate(loo.split(x), 1):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # 테스트 셋의 파일 이름 추출
    test_file_name = image_files[test_index[0]]


    # CNN 모델 생성
    model = Sequential()
    model.add(Conv2D(32, (4, 4), activation='mish', input_shape=(300, 300, 1)))
    model.add(MaxPooling2D((3, 3)))
    model.add(Conv2D(64, (3, 3), activation='mish'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='mish'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(256, (3, 3), activation='mish'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(512, (3, 3), activation='mish'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation=mish))
    model.add(Dense(2, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=15, batch_size=10, validation_split=0.2, verbose=2)

    predictions = model.predict(x_test)
    pred_class = np.argmax(predictions)
    actual_class = np.argmax(y_test)
    acc = (pred_class == actual_class)

    # 분류 결과 출력
    print(f'Fold {fold} - File: {test_file_name}, Predicted Class: {pred_class}, Actual Class: {actual_class}')
    print(f'Fold {fold} Accuracy: {acc}')
    fold_acc.append(acc)

# 모든 폴드의 평균 정확도 출력
accuracy = np.mean(fold_acc)
print(f'Average Accuracy across folds: {accuracy}')


Epoch 1/15
7/7 - 4s - loss: 26.4484 - accuracy: 0.4762 - val_loss: 2.7546 - val_accuracy: 0.4375 - 4s/epoch - 568ms/step
Epoch 2/15
7/7 - 0s - loss: 3.5783 - accuracy: 0.5238 - val_loss: 8.6191 - val_accuracy: 0.5625 - 249ms/epoch - 36ms/step
Epoch 3/15
7/7 - 0s - loss: 1.7180 - accuracy: 0.6349 - val_loss: 0.6984 - val_accuracy: 0.6250 - 246ms/epoch - 35ms/step
Epoch 4/15
7/7 - 0s - loss: 0.8185 - accuracy: 0.7143 - val_loss: 0.9463 - val_accuracy: 0.6250 - 246ms/epoch - 35ms/step
Epoch 5/15
7/7 - 0s - loss: 0.3373 - accuracy: 0.8889 - val_loss: 2.8531 - val_accuracy: 0.4375 - 247ms/epoch - 35ms/step
Epoch 6/15
7/7 - 0s - loss: 0.8757 - accuracy: 0.7619 - val_loss: 0.9329 - val_accuracy: 0.6875 - 248ms/epoch - 35ms/step
Epoch 7/15
7/7 - 0s - loss: 0.2836 - accuracy: 0.8413 - val_loss: 1.0626 - val_accuracy: 0.6250 - 249ms/epoch - 36ms/step
Epoch 8/15
7/7 - 0s - loss: 0.2109 - accuracy: 0.9048 - val_loss: 1.1637 - val_accuracy: 0.6250 - 249ms/epoch - 36ms/step
Epoch 9/15
7/7 - 0s - los

1/1 [==============================] - 0s 97ms/step
Fold 5 - File: x_18.png, Predicted Class: 0, Actual Class: 0
Fold 5 Accuracy: True
Epoch 1/15
7/7 - 2s - loss: 21.3179 - accuracy: 0.4762 - val_loss: 2.8260 - val_accuracy: 0.4375 - 2s/epoch - 356ms/step
Epoch 2/15
7/7 - 0s - loss: 1.4084 - accuracy: 0.5714 - val_loss: 0.6486 - val_accuracy: 0.6875 - 254ms/epoch - 36ms/step
Epoch 3/15
7/7 - 0s - loss: 1.0332 - accuracy: 0.5079 - val_loss: 1.6176 - val_accuracy: 0.4375 - 253ms/epoch - 36ms/step
Epoch 4/15
7/7 - 0s - loss: 0.5515 - accuracy: 0.7937 - val_loss: 1.1527 - val_accuracy: 0.6875 - 290ms/epoch - 41ms/step
Epoch 5/15
7/7 - 0s - loss: 0.5758 - accuracy: 0.8413 - val_loss: 0.3959 - val_accuracy: 0.8125 - 274ms/epoch - 39ms/step
Epoch 6/15
7/7 - 0s - loss: 0.2982 - accuracy: 0.9048 - val_loss: 1.2561 - val_accuracy: 0.6250 - 302ms/epoch - 43ms/step
Epoch 7/15
7/7 - 0s - loss: 0.2845 - accuracy: 0.8889 - val_loss: 1.1758 - val_accuracy: 0.6875 - 289ms/epoch - 41ms/step
Epoch 8/15
7

1/1 [==============================] - 0s 154ms/step
Fold 6 - File: x_17.png, Predicted Class: 0, Actual Class: 0
Fold 6 Accuracy: True
Epoch 1/15
7/7 - 2s - loss: 29.1520 - accuracy: 0.4444 - val_loss: 7.0587 - val_accuracy: 0.4375 - 2s/epoch - 354ms/step
Epoch 2/15
7/7 - 0s - loss: 6.8539 - accuracy: 0.4762 - val_loss: 4.1147 - val_accuracy: 0.4375 - 274ms/epoch - 39ms/step
Epoch 3/15
7/7 - 0s - loss: 1.0891 - accuracy: 0.6032 - val_loss: 0.8654 - val_accuracy: 0.4375 - 245ms/epoch - 35ms/step
Epoch 4/15
7/7 - 0s - loss: 0.5244 - accuracy: 0.7460 - val_loss: 0.6753 - val_accuracy: 0.7500 - 285ms/epoch - 41ms/step
Epoch 5/15
7/7 - 0s - loss: 0.4441 - accuracy: 0.7937 - val_loss: 0.9616 - val_accuracy: 0.7500 - 249ms/epoch - 36ms/step
Epoch 6/15
7/7 - 0s - loss: 0.3936 - accuracy: 0.8254 - val_loss: 1.0705 - val_accuracy: 0.6250 - 248ms/epoch - 35ms/step
Epoch 7/15
7/7 - 0s - loss: 0.3769 - accuracy: 0.8571 - val_loss: 1.5369 - val_accuracy: 0.4375 - 247ms/epoch - 35ms/step
Epoch 8/15


In [ ]:
print(fold_acc)

NameError: name 'fold_acc' is not defined

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(fold_acc) + 1), fold_acc, marker='o', linestyle='-', color='b', label='Fold Accuracy')
plt.axhline(y=average_accuracy, color='r', linestyle='--', label='Average Accuracy')
plt.title('K-Fold Cross Validation Accuracy')
plt.xlabel('Fold')
plt.ylabel('Accuracy')
plt.xticks(range(1, len(fold_acc) + 1))
plt.legend()
plt.grid(True)
plt.show()